In [3]:
#basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import env
from os.path import exists

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Acquire and Summarize

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.  
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method)   
- Only include properties that have a latitude and longitude value.

In [7]:

def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_zillow_data():
    '''
    Reads in all fields from the customers table in the mall_customers schema from data.codeup.com
    
    parameters: None
    
    returns: a single Pandas DataFrame with the index set to the primary customer_id field
    '''

    sql = """
    SELECT 
        prop.*,
        predictions_2017.logerror as logerror,
        aircon.airconditioningdesc as aircon,
        arch.architecturalstyledesc as architecture,
        buildclass.buildingclassdesc as building_class, 
        heating.heatingorsystemdesc as heating,
        landuse.propertylandusedesc as landuse, 
        story.storydesc as story,
        construct_type.typeconstructiondesc as construct_type
    FROM properties_2017 prop
    JOIN (
        SELECT parcelid, MAX(transactiondate) AS max_transactiondate
        FROM predictions_2017
        GROUP BY parcelid
        ) pred USING (parcelid)
    JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                        AND pred.max_transactiondate = predictions_2017.transactiondate
    LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
    LEFT JOIN airconditioningtype aircon USING (airconditioningtypeid)
    LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
    LEFT JOIN buildingclasstype buildclass USING (buildingclasstypeid)
    LEFT JOIN heatingorsystemtype heating USING (heatingorsystemtypeid)
    LEFT JOIN storytype story USING (storytypeid)
    LEFT JOIN typeconstructiontype construct_type USING (typeconstructiontypeid)
    WHERE propertylandusedesc IN ("Single Family Residential", "Inferred Single Family Residential") 
        AND transactiondate like '%%2017%%';
    """

    if exists('zillow_data.csv'):
        df = pd.read_csv('zillow_data.csv')
    else:
        df = pd.read_sql(sql, get_connection('zillow'))
    return df

In [8]:
df = get_zillow_data()

In [9]:
df.fips.value_counts()

6037.0    33823
6059.0    14111
6111.0     4385
Name: fips, dtype: int64

In [14]:
df[df.fips == 6059.0].isnull().mean()

id                          0.000000
parcelid                    0.000000
airconditioningtypeid       0.855786
architecturalstyletypeid    0.995039
basementsqft                1.000000
                              ...   
building_class              1.000000
heating                     0.959039
landuse                     0.000000
story                       1.000000
construct_type              0.994614
Length: 66, dtype: float64

In [4]:
#two brackets to return a dataframe of two columns
# print(df[df.duplicated(subset= 'property_id', keep=False)][['property_id','transactiondate']].sort_values(by=['property_id', 'transactiondate']))
# df[df.duplicated(subset= 'property_id', keep='last')][['property_id','transactiondate']].sort_values(by=['property_id', 'transactiondate'])

In [5]:
df.info() # --> no nulls in lat/longitude value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52319 entries, 0 to 52318
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52319 non-null  int64  
 1   parcelid                      52319 non-null  int64  
 2   airconditioningtypeid         13615 non-null  float64
 3   architecturalstyletypeid      70 non-null     float64
 4   basementsqft                  47 non-null     float64
 5   bathroomcnt                   52319 non-null  float64
 6   bedroomcnt                    52319 non-null  float64
 7   buildingclasstypeid           0 non-null      object 
 8   buildingqualitytypeid         33654 non-null  float64
 9   calculatedbathnbr             52184 non-null  float64
 10  decktypeid                    389 non-null    float64
 11  finishedfloor1squarefeet      4371 non-null   float64
 12  calculatedfinishedsquarefeet  52238 non-null  float64
 13  f

Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)



In [6]:
# examine row by row basics:

def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prnt_miss})
    rows_missing = rows_missing.reset_index().groupby(['num_cols_missing', 'percent_cols_missing']).count().reset_index()

    return rows_missing

In [7]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    percnt_miss = num_missing / df.shape[0] * 100
    cols_missing = pd.DataFrame(
        {
            'num_rows_missing': num_missing,
            'percent_rows_missing': percnt_miss
        }
    )
    return cols_missing

In [8]:
def summarize(df):
    print('-----')
    print('DataFrame info:\n')
    print (df.info())
    print('---')
    print('DataFrame describe:\n')
    print (df.describe())
    print('---')
    print('DataFrame null value asssessment:\n')
    print('Nulls By Column:', nulls_by_col(df))
    print('----')
    print('Nulls By Row:', nulls_by_row(df))
    numerical_cols = df.select_dtypes(exclude='object').columns.to_list()
    categorical_cols = df.select_dtypes(include='object').columns.to_list()
    print('value_counts: \n')
    for col in df.columns:
        print(f'Column Names: {col}')
        if col in categorical_cols:
            print(df[col].value_counts())
        else:
            print(df[col].value_counts(bins=10, sort=False, dropna=False))
            print('---')
    print('Report Finished')
    return

In [9]:
summarize(df)

-----
DataFrame info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52319 entries, 0 to 52318
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52319 non-null  int64  
 1   parcelid                      52319 non-null  int64  
 2   airconditioningtypeid         13615 non-null  float64
 3   architecturalstyletypeid      70 non-null     float64
 4   basementsqft                  47 non-null     float64
 5   bathroomcnt                   52319 non-null  float64
 6   bedroomcnt                    52319 non-null  float64
 7   buildingclasstypeid           0 non-null      object 
 8   buildingqualitytypeid         33654 non-null  float64
 9   calculatedbathnbr             52184 non-null  float64
 10  decktypeid                    389 non-null    float64
 11  finishedfloor1squarefeet      4371 non-null   float64
 12  calculatedfinishedsquarefeet  52238 n

### Takeaways 
* 

Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [10]:
cols_missing = nulls_by_col(df)

In [11]:
print(cols_missing)

                          num_rows_missing  percent_rows_missing
id                                       0              0.000000
parcelid                                 0              0.000000
airconditioningtypeid                38704             73.976949
architecturalstyletypeid             52249             99.866205
basementsqft                         52272             99.910166
...                                    ...                   ...
building_class                       52319            100.000000
heating                              18470             35.302663
landuse                                  0              0.000000
story                                52272             99.910166
construct_type                       52243             99.854737

[66 rows x 2 columns]


### Takeaways 
* Can get rid of the id columns that were used for join. typeconstructionid, storytypeid, etc
* anything below 95% should be dropped
* actually after some research we are going to go with 64%
* want to investigate garagecarcnt, heatingorsystemdesc, airconditiontiontypeid, aircondtioningdesc


# Prepare

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.



#### Takeaways 
* does not appear to be any of those properties in the dataframe
* pulled in only single family residential or inferred single family residential

### Create a function that will drop rows or columns based on the percent of values that are missing: 
`handle_missing_values(df, prop_required_column, prop_required_row)`

> The input:
* A dataframe
* A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column.   
   - i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
* A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row.   
   - For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
> The output:
* The dataframe with the columns and rows dropped as indicated. 
   - Be sure to drop the columns prior to the rows in your function.
   - hint: Look up the dropna documentation.
> You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.


In [12]:
def handle_missing_values(df, prop_required_columns=0.60, prop_required_row=0.75):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)

    return df

In [13]:
df = handle_missing_values(df)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52241 entries, 0 to 52318
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52241 non-null  int64  
 1   parcelid                      52241 non-null  int64  
 2   bathroomcnt                   52241 non-null  float64
 3   bedroomcnt                    52241 non-null  float64
 4   buildingqualitytypeid         33653 non-null  float64
 5   calculatedbathnbr             52180 non-null  float64
 6   calculatedfinishedsquarefeet  52233 non-null  float64
 7   finishedsquarefeet12          52069 non-null  float64
 8   fips                          52241 non-null  float64
 9   fullbathcnt                   52180 non-null  float64
 10  heatingorsystemtypeid         33849 non-null  float64
 11  latitude                      52241 non-null  float64
 12  longitude                     52241 non-null  float64
 13  l

Encapsulate your work inside of functions in a wrangle_zillow.py module.

In [15]:

def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_zillow_data():
    '''
    Reads in all fields from the customers table in the mall_customers schema from data.codeup.com
    
    parameters: None
    
    returns: a single Pandas DataFrame with the index set to the primary customer_id field
    '''

    sql = """
    SELECT 
        prop.*,
        aircon.airconditioningdesc as aircon,
        arch.architecturalstyledesc as architecture,
        buildclass.buildingclassdesc as building_class, 
        heating.heatingorsystemdesc as heating,
        landuse.propertylandusedesc as landuse, 
        story.storydesc as story,
        construct_type.typeconstructiondesc as construct_type
    FROM properties_2017 prop
    JOIN (
        SELECT parcelid, MAX(transactiondate) AS max_transactiondate
        FROM predictions_2017
        GROUP BY parcelid
        ) pred USING (parcelid)
    JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                        AND pred.max_transactiondate = predictions_2017.transactiondate
    LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
    LEFT JOIN airconditioningtype aircon USING (airconditioningtypeid)
    LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
    LEFT JOIN buildingclasstype buildclass USING (buildingclasstypeid)
    LEFT JOIN heatingorsystemtype heating USING (heatingorsystemtypeid)
    LEFT JOIN storytype story USING (storytypeid)
    LEFT JOIN typeconstructiontype construct_type USING (typeconstructiontypeid)
    WHERE propertylandusedesc IN ("Single Family Residential", "Inferred Single Family Residential") 
        AND transactiondate like '%%2017%%';
    """

    if exists('zillow_data.csv'):
        df = pd.read_csv('zillow_data.csv')
    else:
        df = pd.read_sql(sql, get_connection('zillow'))
    return df

In [16]:
# examine row by row basics:

def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prnt_miss})
    rows_missing = rows_missing.reset_index().groupby(['num_cols_missing', 'percent_cols_missing']).count().reset_index()

    return rows_missing

In [17]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    percnt_miss = num_missing / df.shape[0] * 100
    cols_missing = pd.DataFrame(
        {
            'num_rows_missing': num_missing,
            'percent_rows_missing': percnt_miss
        }
    )
    return cols_missing

In [18]:
def summarize(df):
    print('-----')
    print('DataFrame info:\n')
    print (df.info())
    print('---')
    print('DataFrame describe:\n')
    print (df.describe())
    print('---')
    print('DataFrame null value asssessment:\n')
    print('Nulls By Column:', nulls_by_col(df))
    print('----')
    print('Nulls By Row:', nulls_by_row(df))
    numerical_cols = df.select_dtypes(exclude='object').columns.to_list()
    categorical_cols = df.select_dtypes(include='object').columns.to_list()
    print('value_counts: \n')
    for col in df.columns:
        print(f'Column Names: {col}')
        if col in categorical_cols:
            print(df[col].value_counts())
        else:
            print(df[col].value_counts(bins=10, sort=False, dropna=False))
            print('---')
    print('Report Finished')
    return

In [19]:
def handle_missing_values(df, prop_required_columns=0.60, prop_required_row=0.75):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)

    return df

In [20]:
def split_data(df):
    train_validate, test = train_test_split(df, test_size= .2, random_state=514)
    train, validate = train_test_split(train_validate, test_size= .3, random_state=514)
    print(train.shape, validate.shape, test.shape)
    return train, validate, test

In [1]:
def scale_split_data (train, validate, test):
    scaler = MinMaxScaler()

    scaler.fit(train)

    train_scaled = scaler(train)
    validate_scaled = scaler(validate)
    test_scaled = scaler(test)

    return train_scaled, validate_scaled, test_scaled

In [34]:
def prep_zillow (df):
    """ 
    Purpose
        Perform preparation functions on the zillow dataset
    Parameters
        df: data acquired from zillow dataset
    Output
        df: the unsplit and unscaled data with removed columns
        X_train:
        X_train_scaled:
        X_validate:
        X_validate_scaled:
        X_test:
        X_test_scaled:
    """

    #remove unwanted columns, and reset index to id --> for the exercises
    df = df.drop(columns=['parcelid', 'buildingqualitytypeid', 'heatingorsystemtypeid', 'propertylandusetypeid'])
    df.set_index('id')

    #fix data types
    df[['censustractandblock', 'yearbuilt']] = df[['censustractandblock', 'yearbuilt']].astype('str')

    #drop na/duplicates --> adjust this for project. think of columns to impute
    df = df.dropna()

    # take care of any duplicates:
    df = df.drop_duplicates()
    
    #split the data
    train, validate, test = split_data(df)

    #scale the data
    train_scaled, validate_scaled, test_scaled = scale_split_data(train, validate, test)

    return df, train, validate, test, train_scaled, validate_scaled, test_scaled 

In [32]:
def wrangle_zillow():
    """ 
    Purpose
        Perform acuire and preparation functions on the zillow dataset
    Parameters
        None
    Output
        df: the unsplit and unscaled data
        X_train:
        X_train_scaled:
        X_validate:
        X_validate_scaled:
        X_test:
        X_test_scaled:
    """
    #initial data acquisition
    df = get_zillow_data()

    # handle the missing data --> decisions made in advance
    df = handle_missing_values(df, prop_required_columns=0.64)
    
    #drop columns that are unneeded, split data
    df, train, validate, test, train_scaled, validate_scaled, test_scaled = prep_zillow(df)

    #summarize the data
    summarize(df)

    return df, train, validate, test, train_scaled, validate_scaled, test_scaled 

In [35]:
df, train, validate, test, train_scaled, validate_scaled, test_scaled = wrangle_zillow()

(18011, 28) (7720, 28) (6433, 28)
-----
DataFrame info:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32164 entries, 3 to 52318
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            32164 non-null  int64  
 1   bathroomcnt                   32164 non-null  float64
 2   bedroomcnt                    32164 non-null  float64
 3   calculatedbathnbr             32164 non-null  float64
 4   calculatedfinishedsquarefeet  32164 non-null  float64
 5   finishedsquarefeet12          32164 non-null  float64
 6   fips                          32164 non-null  float64
 7   fullbathcnt                   32164 non-null  float64
 8   latitude                      32164 non-null  float64
 9   longitude                     32164 non-null  float64
 10  lotsizesquarefeet             32164 non-null  float64
 11  propertycountylandusecode     32164 non-null  object 
 12  pro

In [2]:
df.fips

NameError: name 'df' is not defined